# 1. Import and Install Dependencies

In [1]:
import cv2
import numpy as np
from tqdm import tqdm
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import glob

# 2. Keypoints using MP Holistic

In [10]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

# 4. Setup Folders for Collection

In [11]:
# Actions that we try to detect
actions = np.array(sorted(os.listdir('dataset/images')))

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 70

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
def get_unify_frames(path):
    """Unify number of frames for each training.
    
    Args:
        path: path to directory.
    """
    offset = 0

    # pick frames
    frames = os.listdir(path)
    frames_count = len(frames)

    if sequence_length > frames_count:
        # duplicate last frame if video is shorter than necessary
        frames += [frames[-1]] * (sequence_length - frames_count)
    elif sequence_length < frames_count:
        # if there are more frames, then sample starting offset
        frames = frames[0:sequence_length]
    return frames

In [12]:
src = 'dataset/images/'
des = 'dataset/data/'

In [ ]:
# for item in actions:
#     n = 'dataset/images/' + item
#     for fol in os.listdir(n):
#         p = des + item
#         os.makedirs(p+'/'+fol)

In [ ]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for item in actions:
        src = 'dataset\\images\\' + item
        for folder in os.listdir(src):
            path = os.path.join(src, folder)
            # print(folder)
            frames = sorted(get_unify_frames(path))
            count = 0
            if len(frames) == sequence_length: # just to be sure
                for frame in frames:
                    frame = cv2.imread(path + '\\' + frame)
                    frame, results  = mediapipe_detection(frame, holistic)
                    if count <= sequence_length:
                        draw_styled_landmarks(frame, results)
                        keypoints = extract_keypoints(results)
                        npy_path = os.path.join(des, item, folder, str(count))
                        print(npy_path)
                        np.save(npy_path, keypoints)
                        count += 1

# 6. Preprocess Data and Create Labels and Features

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label:num for num, label in enumerate(actions)}

In [15]:
label_map

{'backhand': 0,
 'backhand_slice': 1,
 'backhand_volley': 2,
 'flat_service': 3,
 'forehand_flat': 4,
 'forehand_openstands': 5,
 'forehand_slice': 6,
 'forehand_volley': 7,
 'kick_service': 8,
 'slice_service': 9,
 'smash': 10}

In [16]:
sequences, labels = [], []
for action in actions:
    directory = 'dataset/data/' + action
    for sequence in os.listdir(directory):
        print(sequence)
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(des, action, sequence, "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
41
42
43
44
45
46
47
48
49
5
50
51
52
53
54
55
56
57
58
59
6
60
7
8
9
p10_bslice_s1
p10_bslice_s2
p10_bslice_s3
p11_bslice_s1
p11_bslice_s2
p11_bslice_s3
p12_bslice_s1
p12_bslice_s2
p12_bslice_s3
p13_bslice_s1
p13_bslice_s2
p13_bslice_s3
p14_bslice_s1
p14_bslice_s2
p14_bslice_s3
p15_bslice_s1
p15_bslice_s2
p15_bslice_s3
p16_bslice_s1
p16_bslice_s2
p16_bslice_s3
p17_bslice_s1
p17_bslice_s2
p17_bslice_s3
p18_bslice_s1
p18_bslice_s2
p18_bslice_s3
p19_bslice_s1
p19_bslice_s2
p19_bslice_s3
p1_bslice_s1
p1_bslice_s2
p1_bslice_s3
p20_bslice_s1
p20_bslice_s2
p20_bslice_s3
p2_bslice_s1
p2_bslice_s2
p2_bslice_s3
p3_bslice_s1
p3_bslice_s2
p3_bslice_s3
p4_bslice_s1
p4_bslice_s2
p4_bslice_s3
p5_bslice_s1
p5_bslice_s2
p5_bslice_s3
p6_bslice_s1
p6_bslice_s2
p6_bslice_s3
p7_bslice_s1
p7_bslice_s2
p7_bslice_s3
p8_bslice_s1
p8_bslice_s2
p8_bslice_s3
p9_bslice_s1
p9_bslice_s2
p9_bslice_s3
p10_bvolley_s1
p

In [17]:
print(res.shape)
print(len(window))
print(labels)

(132,)
70
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

In [18]:
np.array(sequences).shape

(660, 70, 132)

In [19]:
np.array(labels).shape

(660,)

In [20]:
X = np.array(sequences)

In [21]:
X.shape

(660, 70, 132)

In [22]:
y = to_categorical(labels).astype(int)

In [23]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [25]:
print('X_train = ', X_train.shape)
print('X_test = ', X_test.shape)
print('y_train = ', y_train.shape)
print('y_test = ', y_test.shape)

X_train =  (594, 70, 132)
X_test =  (66, 70, 132)
y_train =  (594, 11)
y_test =  (66, 11)


# 7. Build and Train LSTM Neural Network

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [2]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [3]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(70,132)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(11, activation='softmax'))

In [4]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 70, 32)            21120     
_________________________________________________________________
lstm_1 (LSTM)                (None, 70, 64)            24832     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 11)                363       
Total params: 155,467
Trainable params: 155,467
Non-trainable params: 0
__________________________________________________

In [31]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
19/19 [==============================] - 17s 699ms/step - loss: 2.3992 - categorical_accuracy: 0.0976
Epoch 2/500
19/19 [==============================] - 10s 529ms/step - loss: 2.8160 - categorical_accuracy: 0.0926
Epoch 3/500
19/19 [==============================] - 11s 561ms/step - loss: 2.3956 - categorical_accuracy: 0.1094
Epoch 4/500
19/19 [==============================] - 10s 523ms/step - loss: 2.7945 - categorical_accuracy: 0.1515
Epoch 5/500
19/19 [==============================] - 10s 528ms/step - loss: 2.3967 - categorical_accuracy: 0.0943
Epoch 6/500
19/19 [==============================] - 10s 542ms/step - loss: 2.3955 - categorical_accuracy: 0.0976
Epoch 7/500
19/19 [==============================] - 10s 546ms/step - loss: 2.3940 - categorical_accuracy: 0.0976
Epoch 8/500
19/19 [==============================] - 11s 577ms/step - loss: 2.3880 - categorical_accuracy: 0.1128
Epoch 9/500
19/19 [==============================] - 10s 548ms/step - loss: 2.4399 - cat

KeyboardInterrupt: 

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

# 9. Save Weights

In [ ]:
model.save('action.h5')

In [ ]:
del model

In [6]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# 11. Test in Real Time